Copyright 2022 Jani Kotakoski <jani.kotakoski@iki.fi> and Andreas Postl <andreas.postl@univie.ac.at> 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, XPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


<h4>Preamble

In [ ]:
# Built-in and public packages
import copy
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from math import pi
from lmfit import Model
from periodictable import boron, nitrogen, carbon, elements
from scipy.integrate import simps
from scipy.constants import physical_constants
from scipy import optimize as opt
from scipy import linalg as scipylinalg

# In-house packages
from knockon import calc_vrms, calc_cs

# General constants
h = physical_constants["Planck constant"][0]
hbar = 0.5*h/pi
eV = physical_constants["electron volt"][0]
u = physical_constants["atomic mass constant"][0]
k = physical_constants["Boltzmann constant in eV/K"][0]

# Material constants
a_gra = 0.246 # [nm], lattice constant of graphene
A_unit_cell = a_gra**2 * np.sqrt(3)/2 # [nm^2], area of a unit cell (= inner area of a hexagon)
rho_A = 2/A_unit_cell # [1/nm^2], lattice atoms per unit area

<h4>Helper functions for the analysis and the uncertainty calculation

In [ ]:
def relative_probe_intensity_vs_px(point_source=False, probe_shape_from_file=True, print_info=False):
    
    if point_source == False:
        # Simulate 'scan' with a 2D Gaussian probe
        N_px = 151 # Number of 'pixels' in x and y
                  # Note that a pixel size close to the FWHM (expected: ~ 1.2 Angstroem) is sufficient
        ax = np.linspace(-(N_px-1)*0.5, (N_px-1)*0.5, N_px)
        xx,yy = np.meshgrid(ax, ax)

        ### Probe FWHM = 1 Angstroem, FOV_1D is 1 nm
        ### FWHM in pixels is FWHM/FOX_1D*Npx, and sigma ~ FWHM/2.355
        FOV_1D = 10 # [A]
        px_per_Angstroem = N_px/FOV_1D
        FWHM_expected_px = 1.2*px_per_Angstroem # [pixels]
        sigma_expected_px = FWHM_expected_px/2.355 # [pixels]
        
        if print_info == True:
            print("Expecting a probe FWHM of", FWHM_px/px_per_Angstroem, "A")
            print("Using a pixel distance of", 1/px_per_Angstroem, "A")
    
        probe_shape_from_file = True
        if probe_shape_from_file == True:  
            # ProbeInspector result
            probe_shape_import = np.loadtxt("./table_data/stem_probe_shape.csv", delimiter=",")
            if print_info == True:
                print("Imported probe shape\n", probe_shape_import)
              
            heights = probe_shape_import[:, 0] # relative heights
            sigmas = probe_shape_import[:, 1] # [A]
            sigmas *= px_per_Angstroem # [pixels]
            
            # 2D intensity profile, reconstructed
            probe = np.zeros(xx.shape)
            for height, sigma in zip(heights, sigmas):
                probe += height*np.exp(-(np.square(xx)+np.square(yy))/np.square(sigma))
        else:
            # 2D intensity profile, expected
            probe = np.exp(-0.5*(np.square(xx)+np.square(yy))/np.square(sigma_expected_px))
        probe /= np.sum(probe) # normalization
        
        if print_info == True:
            central_pixel = floor(N_px/2)
            half_of_max_intensity = probe[central_pixel, central_pixel]/2
            FWHM_px = np.searchsorted(probe[central_pixel, central_pixel::-1] \
                                      < half_of_max_intensity, 1)*2 # [pixels]
            FWHM = FWHM_px/px_per_Angstroem # [A]
            print("Reconstructed probe FWHM: ", FWHM, "[A]")
            
        probe_vs_px = probe.reshape((probe.size, )) # reshape probe to 1D
    else:
        probe_vs_px = np.array([1])
        
    return probe_vs_px
    
def scan_fraction(probe_vs_px):
    # Function to limit the actual observation time to a fraction of t_frame
    
    # Pixels to scan until observation (for each possible vacancy position)
    i = np.array([(int(0.5*probe_vs_px.size) + i)%int(probe_vs_px.size) for i in range(probe_vs_px.size)])+1
    # fraction of scan (limits the time)
    p = i/probe_vs_px.size
    
    return p

def nu(c_ad, N_v=1.6):
    # Number of steps necessary to reach a vacancy
    # Precisely: The expectation value of the exponential distribution
    
    bonds_per_atom = 3/2
    return - 1/N_v * np.log(1-c_ad/bonds_per_atom)

def P_obs(E_m, c_ad_A, T_array, f0, probe_vs_px):
    # Calculation of the probability to actually observe a vacancy from the expontential distribution
    # P_obs = 1 - P_h, P_h ... Probability for healing
    
    try:
        iter(T_array)
    except:
        T_array = np.array([T_array])
    
    t_frame = 0.5 # [s] typical frame time
    P_obs = np.empty(T_array.shape)
    for i, T in enumerate(T_array):
        # Probability for actually meeting the vacancy from exponential distribution
        exponent = -(t_frame*f0*np.exp(-E_m/(k*T))*nu(c_ad_A/rho_A, N_v=1.6))
        exponent *= scan_fraction(probe_vs_px)
        summands = np.exp(exponent)*probe_vs_px
        P_obs[i] = np.sum(summands)

    return P_obs
    
def sigma_obs_ko_barn(E_kin, T_array, E_m, c_ad_A, f0, probe_vs_px): # [b]
    # E_kin in eV
    # T in K
    # E_m in eV
    # c_ad in 1/nm^2
    # f0 in Hz
    
    try:
        iter(T_array)
    except:
        T_array = np.array([T_array])
        
    sigma_obs_ko = np.empty(T_array.shape)
    for i, T in enumerate(T_array):
        vrms = vrms_factor*calc_vrms(dos_gr[:,0], 0.5*dos_gr[:,1], carbon.mass, T)**0.5
        sigma_obs_ko[i] = calc_cs(carbon.mass, carbon.number, Td, E_kin, vrms) \
                          *P_obs(E_m, c_ad_A, T, f0, probe_vs_px)
    return sigma_obs_ko*1e28 # [b], "observed" knock-on damage cross section

def calculate_curvature(x, y, x0):
    # This function computes the local curvature of y=f(x) at x0.
    # This corresponds to the matrix element of a 1x1 Hessian,
    # which is the matrix containing the 2nd derivatives of the cost function w.r.t. the fit parameters.
    # The variance-covariance matrix of fit parameters is given by pcov = s_sq * inv(1/2*H),
    # where s_sq is the residual variance and inv(1/2*H) the inverse of the Hessian divided by two.
    #
    # x ... array-like, must contain equidistant values
    
    idx_x0 = np.argmin(np.abs(x - x0))
    dx = x[1]-x[0]
    dydx = np.gradient(y, dx)
    dy2dx2 = np.gradient(dydx, dx)
    
    return dy2dx2[idx_x0]

<h4>Read phonon density of states (from former studies)

In [ ]:
# Supplementary Material attached to Susi et al., Nat Commun 7, 13040 (2016). DOI: 10.1038/ncomms13040
dos_gr = np.loadtxt("./table_data/graphene_zdos_ncomms.dat") # x in eV
dos_gr[:,0] *= eV
dos_gr[:,0] /= hbar
dos_int = simps(dos_gr[:,1], dos_gr[:,0])
dos_gr[:,1] /= (dos_int/2)

<h4>Read experimental data

In [ ]:
experiment = pd.read_csv("./table_data/table_knockon_cs_90kv.csv",
                         sep =',',
                         header = 0,
                         index_col = 0,
                         quotechar = '"',
                         decimal = '.')

# Fix data import of lists
experiment["mean beam current [pA]"] = [eval(i) for i in experiment["mean beam current [pA]"]]
experiment["total acquisition time [s]"] = [eval(i) for i in experiment["total acquisition time [s]"]]

# Variable aliases
mean_electron_flux = [i[0] for i in experiment["mean beam current [pA]"]][0]*1e-12/eV # [electrons/s]
Delta_mean_electron_flux = [i[1] for i in experiment["mean beam current [pA]"]][1]*1e-12/eV # [electrons/s]

# Display data
display(experiment)

<h4>Theoretical knock-on cross-section

In [ ]:
Td = 21.03 # [eV], new "best fit" value is Td = 21.03 +- 0.10
vrms_factor = 1.05 # Relative to the result obtained from phonon-DOS
Delta_Td, Delta_vrms_factor = 0.065, 0 # Manually adjusted value here;
                                       # This makes up for omitting cov(Td, vrms) with a
                                       # Pearson-Spearson correlation of 0.85 for cov(Td, vrms)

## Cross section as a function of temperature, at 90 keV
E_kin = 90e3 # [eV], Electron kinetic energy
T_array = np.linspace(150, 1200, 250)

# Calculate absolute root-mean-square velocity as a function of temperature
vrms = vrms_factor*calc_vrms(dos_gr[:,0], 0.5*dos_gr[:,1], carbon.mass, T_array)**0.5
vrms_minus, vrms_plus = vrms*(1-Delta_vrms_factor), vrms*(1+Delta_vrms_factor)
 
# Calculate cross section values and uncertainties as a function of temperature
sigma_theor_ko = calc_cs(carbon.mass, carbon.number, Td, E_kin, vrms)
sigma_theor_ko_minus = calc_cs(carbon.mass, carbon.number, Td+Delta_Td, E_kin, vrms_minus)
sigma_theor_ko_plus = calc_cs(carbon.mass, carbon.number, Td-Delta_Td, E_kin, vrms_plus)
    
# Cross section as a function of electron kinetic energy, at lab temperature
T = 300 # [K], Lab room temperature
E_kin_array = np.linspace(10e3, 105e3, 20)

vrms = calc_vrms(dos_gr[:,0], 0.5*dos_gr[:,1], carbon.mass, T)**0.5
sigma_theor_ko_E = calc_cs(carbon.mass, carbon.number, Td, E_kin_array, vrms)


# Plots (for debug purposes only)
debug = False
if debug == True:
    plt.figure()
    plt.plot(T_array, sigma_theor_ko)
    plt.xlabel("T [K]")
    plt.ylabel("cs [m^2]")
    plt.figure()
    plt.xlabel("E [eV]")
    plt.ylabel("cs [m^2]")
    plt.plot(E_kin_array, sigma_theor_ko_E)

<h4>Weightted NLLSQ (nonlinear least squares) fit

In [ ]:
def weighted_residual_variance(sigma_obs_ko_model, sigma_obs_ko_exp):
    #weights = (1/Delta_sigma_obs_ko_exp)**2 # Weighting: Squared inverse uncertainty  
    #weights = np.ones(len(Delta_sigma_obs_ko_exp)) # Flat weighting
    weights = (sigma_obs_ko_exp/Delta_sigma_obs_ko_exp)**2 # Weighting: Squared relative uncertainties
    out = 0
    for i in range(len(sigma_obs_ko_model)):
        out += (sigma_obs_ko_model[i]-sigma_obs_ko_exp[i])**2 * weights[i]
    return out/(len(sigma_obs_ko_model)-2) # Two parameters (E_m, c_ad) lower the degrees of freedom by 2

# Experimental data
T_exp_valid = np.array(experiment["Temperature [K]"])
sigma_obs_ko_exp = np.array(experiment["sigma [mb]"])
Delta_sigma_obs_ko_exp = np.array(experiment["Delta_sigma (statistical) [mb]"])

# For optimization, edit values here
f0 = 4e12 # [1/s]
probe_vs_px = relative_probe_intensity_vs_px(point_source=False,
                   probe_shape_from_file=True, print_info=False) # Flattened probe intensity array
E_m_array = np.linspace(0.3, 0.35, 50) # [eV]
c_ad_A_array = np.linspace(1.9e-6, 2.3e-6, 50) # [1/nm2]
s_sq_array = np.empty((E_m_array.size, c_ad_A_array.size))

# (A) Perform NLLSQ calculation for both E_m and c_ad_A
for i, Enow in enumerate(E_m_array):
    for j, cnow in enumerate(c_ad_A_array):
        sigma_obs_ko_model = sigma_obs_ko_barn(90e3, T_exp_valid, Enow, cnow, f0, probe_vs_px)*1e3 # [mb]
        s_sq_array[i, j] = weighted_residual_variance(sigma_obs_ko_model, sigma_obs_ko_exp)
    
# (B) Perform NLLSQ calculation for c_ad_A at E_m = 0.41 (the average DFT result) 
E_m_DFT = 0.41
c_ad_A_DFT_array = np.linspace(15e-6, 25e-6, 50)
s_sq_DFT_array = np.empty((c_ad_A_DFT_array.size, 1))
for i, cnow in enumerate(c_ad_A_DFT_array):
    sigma_obs_ko_model = sigma_obs_ko_barn(90e3, T_exp_valid, E_m_DFT, cnow, f0, probe_vs_px)*1e3 # [mb]
    s_sq_DFT_array[i] = weighted_residual_variance(sigma_obs_ko_model, sigma_obs_ko_exp)

In [ ]:
# Find NLLSQ values for (A), both E_m and c_ad_A
ij_lsq = np.where(s_sq_array == np.min(s_sq_array))
i_lsq = ij_lsq[0][0]
j_lsq = ij_lsq[1][0]
E_m_lsq = E_m_array[i_lsq]
c_ad_A_lsq = c_ad_A_array[j_lsq]
print("E_m (weighted LSQ solution, nonlinear regression):", E_m_lsq, "eV")
print("c_ad_A:", c_ad_A_lsq, "1/nm²")
print("1D-FOV (length) per adatom:", np.sqrt(1/c_ad_A_lsq))

# Uncertainty calculation
sigma_obs_ko_model_lsq = sigma_obs_ko_barn(90e3, T_exp_valid, E_m_lsq, c_ad_A_lsq, f0, probe_vs_px)*1e3 # [mb]
s_sq = weighted_residual_variance(sigma_obs_ko_model_lsq, sigma_obs_ko_exp) # residual variance at LSQ solution
s_sq_control_value = s_sq_array[i_lsq, j_lsq] # control value for residual variance
print("Residual/Error variance at LSQ solution:", s_sq)
print("Residual/Error variance (control value):", s_sq_control_value)

C = s_sq_array[:, j_lsq] # cost function of variations in E_m at constant c_ad_A
H = calculate_curvature(E_m_array, C, E_m_lsq) # matrix element of the Hessian
pcov = np.linalg.inv(1/2*np.array(H, ndmin=2))*s_sq # variance of the fit parameter
Delta_E_m_lsq = np.sqrt(pcov)[0][0] # sqrt of variance = uncertainty estimate
print("E_m error estimate:", Delta_E_m_lsq)

# Find adatom concentration value for (B), when E_m was fixated at the DFT result 
i_DFT_lsq = np.where(s_sq_DFT_array == np.min(s_sq_DFT_array))[0][0]
c_ad_A_DFT_lsq = c_ad_A_DFT_array[i_DFT_lsq]
print("--------------------------------")
print("Regarding the DFT-result for the barrier:")
print("c_ad_A_DFT:", c_ad_A_DFT_lsq, "1/nm²")
print("1D-FOV (length) per adatom:", np.sqrt(1/c_ad_A_DFT_lsq))
sigma_obs_ko_model_DFT_lsq = sigma_obs_ko_barn(90e3, T_exp_valid, E_m_DFT, c_ad_A_DFT_lsq, f0, probe_vs_px)*1e3 # [mb]
s_sq_DFT = weighted_residual_variance(sigma_obs_ko_model_DFT_lsq, sigma_obs_ko_exp) # residual variance at DFT solution
print("Residual/Error variance at DFT solution:", s_sq_DFT)

In [ ]:
# Visualization of the LSQ fit
plt.figure(figsize=(10, 8))
plt.imshow(s_sq_array, aspect="auto", #vmin=2480, vmax=3700,
           extent = (min(c_ad_A_array), max(c_ad_A_array), max(E_m_array), min(E_m_array)))
plt.colorbar()
plt.ylabel('$E_\mathrm{m}$ [eV]')
plt.xlabel('$c_\mathrm{ad}^\mathrm{A}$ [1/nm2]')
plt.ticklabel_format(axis="x", style="scientific", scilimits=(0,0))
plt.title("Weighted residual variance")
plt.grid()
plt.show()

<h4>Re-evaluation of the model with fit parameters, DFT values, and DFT w/o finite probe

In [ ]:
## Dependent variables: migration energy barrier, adatom concentration, attempt frequency
# Sets of dependent variables; Values are obtained below/elsewhere (Fit below, DFT elsewhere)
E_m_array_result = np.array([0.3265, 0.4100]) # [eV], migration energy barrier
c_ad_A_array_result = np.array([1.973e-6, 2.031e-5]) # [1/nm^2], adatom concentration
f0_array_result = 4e12*np.ones(c_ad_A_array_result.shape) # [Hz], attempt rate
s_sq_array_result = np.array([s_sq, s_sq_DFT]) # weighted least-squares sums from fits (see above)

## Calculation the numerical values of the extended healing model
## for different pairs of migration energies and concentrations
P_obs_dict = {}
P_obs_plus_dict = {}
P_obs_minus_dict = {}
Delta_rel_c_ad_A_array_result = 0.45 * s_sq_array_result/s_sq_array_result[0] 
    # The relative concentration uncertainty for the adatom concentration in the NLLSQ fit was set to ~ 45 %,
    # so that the numerical criterion Delta_relative_sigma_obs_ko(1073 K) = 20% is met. See also (**)

# With the reconstructed finite probe
for i, (E_m, c_ad_A, Delta_rel_c_ad_A, f0) in \
        enumerate(zip(E_m_array_result, c_ad_A_array_result, Delta_rel_c_ad_A_array_result, f0_array_result)):
    
    dict_key = str(E_m)+" eV, "+str(f0)+" 1/s"
    P_obs_dict[dict_key] = np.empty(T_array.shape)
    P_obs_plus_dict[dict_key] = np.empty(T_array.shape)
    P_obs_minus_dict[dict_key] = np.empty(T_array.shape)
    
    P_obs_dict[dict_key] = P_obs(E_m, c_ad_A, T_array, f0, probe_vs_px)
    P_obs_plus_dict[dict_key] = P_obs(E_m, c_ad_A*(1-Delta_rel_c_ad_A), T_array, f0, probe_vs_px)
    P_obs_minus_dict[dict_key] = P_obs(E_m, c_ad_A*(1+Delta_rel_c_ad_A), T_array, f0, probe_vs_px)
    
# With a point-shaped probe
point_probe = relative_probe_intensity_vs_px(point_source=True)
for i, (E_m, c_ad_A, Delta_rel_c_A, f0) in \
        enumerate(zip(E_m_array_result, c_ad_A_array_result, Delta_rel_c_ad_A_array_result, f0_array_result)):

    dict_key = str(E_m)+" eV, "+str(f0)+" 1/s (w/o finite probe)"    
    P_obs_dict[dict_key] = np.empty(T_array.shape)
    P_obs_plus_dict[dict_key] = np.empty(T_array.shape)
    P_obs_minus_dict[dict_key] = np.empty(T_array.shape)
    
    P_obs_dict[dict_key] = P_obs(E_m, c_ad_A, T_array, f0, point_probe)
    P_obs_plus_dict[dict_key] = P_obs(E_m, c_ad_A*(1-Delta_rel_c_ad_A), T_array, f0, point_probe)
    P_obs_minus_dict[dict_key] = P_obs(E_m, c_ad_A*(1+Delta_rel_c_ad_A), T_array, f0, point_probe)
    
# (**) User feedback regarding the uncertainty of the adatom concentration
idx_1073K = np.argmin(np.abs(T_array-1073.15))
dict_key_lsq = list(P_obs_dict.keys())[0]

Delta_rel_c_ad_A_posterior = \
    (P_obs_plus_dict[dict_key_lsq][idx_1073K] - P_obs_dict[dict_key_lsq][idx_1073K])\
    /P_obs_dict[dict_key_lsq][idx_1073K]
print("Relative error of adatom concentration at 1073 K (= 800 C):", Delta_rel_c_ad_A_posterior)

<h4>Figure with publication quality

In [ ]:
# Plot style
plt.rcParams.update({
    "figure.figsize": (16, 12),
    "text.usetex": True,
    "font.family": "serif",
    "axes.linewidth": 3,
    "font.size": 30,
    "lines.markeredgewidth": 2,
    "lines.linewidth": 3,
    "lines.markersize": 5,
    "errorbar.capsize": 5,
    "legend.framealpha": 1,
    #"font.sans-serif": ["Helvetica"]
    })
font = {'family': 'serif',
        'weight': 'bold',
        'size': 42,
        }
font_legend = copy.deepcopy(font)
font_legend['size'] = 24

# Pure knock-on model without vacancy healing
fig = plt.figure()
plt.plot(T_array, 1e31*sigma_theor_ko, 'g-', label = '$\sigma_\mathrm{ko}^\mathrm{theor}$')
plt.fill_between(T_array, 1e31*sigma_theor_ko_plus, 1e31*sigma_theor_ko_minus,
                 facecolor='green', color='green', alpha=0.15)

# Healing model
alphas = [0.85, 0.85, 0.55, 0.55]
colors = [(255/255,100/255,0), (90/255,0,150/255)]*2
label_prepend = ["(LSQ fit)",
                 "(DFT)",
                 "(LSQ fit) w/o finite probe",
                 "(DFT) w/o finite probe"]
line_style = ["-", "-", "--", "--"]
dict_keys = list(P_obs_dict.keys())
#print(dict_keys)

indices_to_plot = [0, 1, 3]

for j in range(len(dict_keys)):        
        i = 3-j
        if i in indices_to_plot:
            dict_key = dict_keys[i]
        else:
            continue
               
        label = "$E_\mathrm{m}$" + f" = {E_m_array_result[i%2]:.2f} eV" + " " + label_prepend[i]
        
        plt.plot(T_array, 1e31*sigma_theor_ko*P_obs_dict[dict_key], \
                 line_style[i], color=colors[i], label=label, alpha=alphas[i])
        
        plt.fill_between(T_array, 1e31*sigma_theor_ko_plus*P_obs_plus_dict[dict_key], \
                                  1e31*sigma_theor_ko_minus*P_obs_minus_dict[dict_key], \
                         facecolor=colors[i], color=colors[i], alpha=.15)
        
# Experimental data
LL = [True, True, True, True, True, True, True, True]
xx = np.array(experiment["Temperature [K]"])
xx = xx[LL]
yy = np.array(experiment["sigma [mb]"])
yy = yy[LL]
xxerr = xx*.02 # 2 % uncertainty for the temperature value
yyerr_upper = np.array(experiment["Delta_sigma (statistical) [mb]"])
yyerr_upper = yyerr_upper[LL]
yyerr_lower = np.array(experiment["Delta_sigma (statistical) [mb]"])
yyerr_lower = yyerr_lower[LL]
yyerr = [yyerr_upper, yyerr_lower]
plt.errorbar(xx, yy, fmt='ro', xerr=xxerr, yerr=yyerr, label='experimental data')

# Plot appearance
ylim_up = 20
plt.xlabel('$T$ [K]', fontdict=font)
plt.ylabel("$\sigma_\mathrm{ko}$ [mb]", fontdict=font)
plt.xlim([200, 1200])
plt.ylim([0, ylim_up])
#plt.legend(prop=font_legend, loc='upper right')
ax = plt.gca()
ax.set_xticks(np.arange(200, 1201, 200))
ax.set_xticks(np.arange(200, 1201, 50), minor=True)
ax.set_yticks(np.arange(0, ylim_up*1.0000001, 5))
ax.set_yticks(np.arange(0, ylim_up*1.0000001, 2.5), minor=True)
ax.tick_params(axis="both", length=16, width=3, direction="inout")
ax.grid(True, which='both')
ax.grid(which="major", alpha=0.6)
ax.grid(which="minor", alpha=0.3)
ax.legend(bbox_to_anchor=(1, 1));

#fig.savefig('plot_cross_section_full_model.pdf', bbox_inches="tight")